## Analysis

<l>
    <li> Find best hyperparameters.</li>
    <li> Plot per class tables for best models.</li>
<l>

In [1]:
import numpy as np
import pandas as pd

import qgrid

# import dependencies
import nltk
import json
import io
import gzip
import torch
import string
import random
import jsonlines
import pandas as pd
import pickle as pkl
import numpy as np
from tqdm import tqdm
from functools import partial

import torch
import torch.nn as nn
import torch.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, RandomSampler, SequentialSampler, DataLoader
from sklearn.preprocessing import MultiLabelBinarizer

import importlib

import utils
# importlib.reload(utils)
from preprocess import create_lookups_for_vocab, pad_collate_fn

import model
# importlib.reload(model)
from model import FinalModel
from torchcontrib.optim import SWA

[nltk_data] Downloading package stopwords to /home/mz2476/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
device = "cpu" # "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
PATH_TO_EMBEDDINGS_FOLDER = "/scratch/mz2476/wiki/embeddings/"
PATH_TO_DATA_FOLDER = "/scratch/mz2476/wiki/data/"
PATH_TO_MODELS_FOLDER = "/scratch/mz2476/wiki/models/"

In [6]:
def load_data_and_create_dataloaders(filename_vocab, filename_wiki_tensor_dataset, filename_classes):
    # LOAD vocab, tensor dataset, classes
    vocab = torch.load(PATH_TO_DATA_FOLDER + filename_vocab)
    print("Vocab size is:", len(vocab))
    index_to_word, word_to_index = create_lookups_for_vocab(vocab)

    wiki_tensor_dataset = torch.load(PATH_TO_DATA_FOLDER + filename_wiki_tensor_dataset)

    classes = torch.load(PATH_TO_DATA_FOLDER + filename_classes)
    mlb = MultiLabelBinarizer(classes)

    # create dataloader
    wiki_loaders = {}

    batch_size = 32

    for split, wiki_dataset in wiki_tensor_dataset.items():
        wiki_loaders[split] = DataLoader(
            wiki_dataset, 
            batch_size=batch_size, 
            shuffle=True, 
            collate_fn=partial(pad_collate_fn, word_to_index=word_to_index)
        )
    return vocab, index_to_word, word_to_index, wiki_tensor_dataset, classes, wiki_loaders

### Load aggregated results to find best model

In [5]:
results = {}
results["results_1_layer"] = pd.read_csv("results/results_tuning_SWA.csv", index_col=0)

In [ ]:
# results["results_1_layer"]["max_num_tokens"] = None
# results["results_2_3_layers"]["max_num_tokens"] = None
# results["results_1_layer_maxlen_500"]["max_num_tokens"] = 500
# results["results_2_3_layers_maxlen_500"]["max_num_tokens"] = 500

In [7]:
df_results = pd.concat(results.values(), ignore_index=True)

In [8]:
# jupyter nbextension enable --py --sys-prefix qgrid
import qgrid
# only required if you have not enabled the ipywidgets nbextension yet
# jupyter nbextension enable --py --sys-prefix widgetsnbextension
#to show a df simply use the below:
qgrid.show_grid(df_results)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Load best model, show per class tables for it

In [6]:
vocab, index_to_word, word_to_index, wiki_tensor_dataset, classes, wiki_loaders = \
    load_data_and_create_dataloaders("vocab_all_en.pt", "wiki_tensor_dataset_vocab_all_en.pt", "classes_list.pt")

Vocab size is: 682850


In [10]:
# import utils
# import importlib
# importlib.reload(utils)

# # Aligned fasstext. 2.5 million
embeddings = utils.load_vectors(PATH_TO_EMBEDDINGS_FOLDER + "wiki.en.align.vec")
#Creating the weight matrix for pretrained word embeddings
weights_matrix_ve = utils.create_embeddings_matrix(word_to_index, embeddings)

341345it [00:26, 13650.51it/s]

KeyboardInterrupt: 

In [ ]:
options_best_1_layer = {
    "VOCAB_SIZE": len(index_to_word),
    "dim_e": weights_matrix_ve.shape[1],
    "pretrained_embeddings": weights_matrix_ve,
    "num_layers": 1,
    "num_classes": len(classes),
    "mid_features": 150,
    "dropout_rate": 0.2,
    "activation": nn.ReLU(),
    "file_name": "en_optimizer_SWA_num_hidden_1_dim_hidden_150_dropout_rate_0_learning_rate_0.01_num_epochs_10.pth"
}

options_best = {
    "VOCAB_SIZE": len(index_to_word),
    "dim_e": weights_matrix_ve.shape[1],
    "pretrained_embeddings": weights_matrix_ve,
    "num_layers": 2,
    "num_classes": len(classes),
    "mid_features": 200,
    "dropout_rate": 0.2,
    "activation": nn.ReLU(),
    "file_name": "en_optimizer_SWA_num_hidden_2_dim_hidden_200_dropout_rate_0.2_learning_rate_0.01_num_epochs_10.pth"
}

In [ ]:
options = options_best_2

model = FinalModel(options)

file_name = options["file_name"]
model.load_state_dict(torch.load(
    f"{PATH_TO_MODELS_FOLDER}/{file_name}",
    map_location=torch.device('cpu')
))
model.to(device)

In [ ]:
# ls $PATH_TO_MODELS_FOLDER

2 layer

In [39]:
# metrics_dict = utils.test_model(wiki_loaders["val"], model, device, threshold=0.5)

# df_per_class_metrics = utils.create_per_class_tables(wiki_loaders["val"], model, device, classes, threshold=0.5)
# df_per_class_metrics.to_csv("results/per_class_metrics_val_best_2_layers_model.csv")

/home/mz2476/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [149]:
df_per_class_metrics = pd.read_csv("results/per_class_metrics_val_best_2_layers_model.csv", index_col=0)
qgrid.show_grid(df_per_class_metrics)
# df_per_class_metrics

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

1 layer

In [46]:
# metrics_dict = utils.test_model(wiki_loaders["val"], model, device, threshold=0.5)

# df_per_class_metrics = utils.create_per_class_tables(wiki_loaders["val"], model, device, classes, threshold=0.5)
# df_per_class_metrics.to_csv("results/per_class_metrics_val_best_1_layer_model.csv")

In [52]:
df_per_class_metrics = pd.read_csv("results/per_class_metrics_val_best_1_layer_model.csv", index_col=0)
df_per_class_metrics

,class_name,count,TN,FN,TP,FP,precision,recall,f1
0,Culture.Arts,19.0,9977,15,4,0,1.000000,0.210526,0.347826
1,Culture.Broadcasting,217.0,9739,104,113,40,0.738562,0.520737,0.610811
2,Culture.Crafts and hobbies,14.0,9982,14,0,0,0.000000,0.000000,0.000000
3,Culture.Entertainment,295.0,9659,131,164,42,0.796117,0.555932,0.654691
4,Culture.Food and drink,67.0,9925,32,35,4,0.897436,0.522388,0.660377
5,Culture.Games and toys,109.0,9882,41,68,5,0.931507,0.623853,0.747253
6,Culture.Internet culture,6.0,9990,6,0,0,0.000000,0.000000,0.000000
7,Culture.Language and literature,3631.0,5993,421,3210,372,0.896147,0.884054,0.890060
8,Culture.Media,3.0,9993,3,0,0,0.000000,0.000000,0.000000
9,Culture.Music,435.0,9472,92,343,89,0.793981,0.788506,0.791234


## Analysis of models for Russian articles

Load data and embedding matrix

In [7]:
vocab, index_to_word, word_to_index, wiki_tensor_dataset, classes, wiki_loaders = \
    load_data_and_create_dataloaders("vocab_all_ru.pt", "wiki_tensor_dataset_vocab_all_ru.pt", "classes_list.pt")

Vocab size is: 376365


In [11]:
# Load weights matrix
weights_matrix_ve = torch.load(PATH_TO_DATA_FOLDER + "embedding_weights_matrix_vocab_all_ru.pt")

# # Load embeddings
# embeddings = utils.load_vectors(PATH_TO_EMBEDDINGS_FOLDER + "wiki.ru.align.vec")
# # Create the weight matrix for pretrained word embeddings
# weights_matrix_ve = utils.create_embeddings_matrix(word_to_index, embeddings)

In [12]:
import model
import importlib
importlib.reload(model)

from model import FinalModel
from torchcontrib.optim import SWA

In [13]:
# load models
dict_model_names = {
    "frozen": {
        "file_name": "ru_optimizer_SWA_num_hidden_2_dim_hidden_150_dropout_rate_0.2_learning_rate_0.01_num_epochs_10_frozen.pth",
    },
    "finetuned": {
        "file_name": "ru_optimizer_SWA_num_hidden_2_dim_hidden_150_dropout_rate_0.2_learning_rate_0.01_num_epochs_10_init_pretrained.pth",   
    },
    "trained": {
        "file_name": "ru_optimizer_SWA_num_hidden_2_dim_hidden_150_dropout_rate_0.2_learning_rate_0.01_num_epochs_10.pth",   
    },
}

options = {
    "VOCAB_SIZE": len(index_to_word),
    "dim_e": weights_matrix_ve.shape[1],
    "pretrained_embeddings": weights_matrix_ve,
    "num_layers": 2,
    "num_classes": len(classes),
    "mid_features": 150,
    "dropout_rate": 0.2,
    "activation": nn.ReLU(),
}

for model_name in dict_model_names.keys():
    model = FinalModel(options)
    # load the state dict from file
    file_name = dict_model_names[model_name]["file_name"]
    model.load_state_dict(torch.load(
        f"{PATH_TO_MODELS_FOLDER}{file_name}",
        map_location=torch.device('cpu')
    ))
    model.to(device)
    # save model to dict
    dict_model_names[model_name]["model"] = model

In [14]:
# Calculate per class tables

from utils import test_model

for model_name in dict_model_names.keys():
    model = dict_model_names[model_name]["model"]
    # print aggregated metrics
    metrics_dict = test_model(wiki_loaders["val"], model, device=device)
    metrics_dict = {key: round(value, 4) for key, value in metrics_dict.items()}
    print("---", model_name)
    print("Precision macro: {}, Recall macro: {}, F1 macro: {} ".format(
        metrics_dict["precision_macro"], metrics_dict["recall_macro"], metrics_dict["f1_macro"]
    ))
    print("Precision micro: {}, Recall micro: {}, F1 micro: {} ".format(
        metrics_dict["precision_micro"], metrics_dict["recall_micro"], metrics_dict["f1_micro"]
    ))
    
    # save per class tables
    df_per_class_metrics = utils.create_per_class_tables(
        wiki_loaders["val"], model, device, classes, threshold=0.5
    )
    dict_model_names[model_name]["df_results"] = df_per_class_metrics
#     # SAVE to file
#     df_per_class_metrics.to_csv(f"results/ru_per_class_metrics_val_{model_name}.csv")

/home/mz2476/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/mz2476/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


--- frozen
Precision macro: 0.3503, Recall macro: 0.1435, F1 macro: 0.1715 
Precision micro: 0.7678, Recall micro: 0.2693, F1 micro: 0.3987 
--- finetuned
Precision macro: 0.6015, Recall macro: 0.4704, F1 macro: 0.516 
Precision micro: 0.8187, Recall micro: 0.7468, F1 micro: 0.7811 
--- trained
Precision macro: 0.5225, Recall macro: 0.3148, F1 macro: 0.3643 
Precision micro: 0.8348, Recall micro: 0.6714, F1 micro: 0.7443 


In [21]:
# df_per_class_metrics = utils.create_per_class_tables(
#         wiki_loaders["train"], dict_model_names["frozen"]["model"], device, classes, threshold=0.5
#     )
print("frozen")
qgrid.show_grid(df_per_class_metrics)

frozen


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [16]:
model_name = list(dict_model_names.keys())[0]
print(model_name)
qgrid.show_grid(dict_model_names[model_name]["df_results"])

frozen


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [17]:
model_name = list(dict_model_names.keys())[1]
print(model_name)
qgrid.show_grid(dict_model_names[model_name]["df_results"])

finetuned


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [18]:
model_name = list(dict_model_names.keys())[2]
print(model_name)
qgrid.show_grid(dict_model_names[model_name]["df_results"])

trained


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

**Why frozen model classifies Russian articles on `Culture.Language and literature` incorrectly?**
- Distribution is different: in Russian articles most (3762/4178) articles within `Culture.Language and literature` are about People. Maybe in English it's not the case? 
    - Isaac said the distribution in English is very similar.
- look at such articles in English vs. in Russian
    - word категория (category) is present many times for each article (for any label)
    - relativley short? no -- they are longer than on average.
    - Have many dates? one nine zero zero ... 
    - Look at predicitons:

In [25]:
df_wiki_ru = torch.load(PATH_TO_DATA_FOLDER + "df_wiki_train_ru.pt")
df_wiki_en = torch.load(PATH_TO_DATA_FOLDER + "df_wiki_train_en.pt")

In [16]:
class2idx = {name: idx for idx, name in enumerate(classes)}
idx2class = classes

In [17]:
class2idx['Culture.Language and literature']

7

In [20]:
from functools import partial

def has_label_num(vec, label_num):
    return int(vec[label_num] == 1)

def has_label(labels_list, label):
    return int(label in set(labels_list))

In [43]:
df_wiki_ru["Culture.Language and literature"] = df_wiki_ru.labels.apply(partial(has_label_num, label_num=7))
df_wiki_ru["Culture.People"] = df_wiki_ru.mid_level_categories_initial.apply(partial(has_label, label="Culture.People"))

In [24]:
print("Number of aritcles that have label Culture.Language and literature:", df_wiki_ru[df_wiki_ru["Culture.Language and literature"] == 1].shape[0])
print("Number of aritcles that have label Culture.People:", df_wiki_ru[df_wiki_ru["Culture.People"] == 1].shape[0])

Number of aritcles that have label Culture.Language and literature: 4178
Number of aritcles that have label Culture.People: 3762


In [53]:
# load "Culture.Language and literature" articles for en and ru
df_language_and_literature_ru = df_wiki_ru[df_wiki_ru["Culture.Language and literature"] == 1]
mask = df_wiki_en.labels.apply(partial(has_label_num, label_num=class2idx["Culture.Language and literature"]))
df_language_and_literature_en = df_wiki_en[mask == 1]

In [70]:
idx = 8900
df_wiki_ru.iloc[idx]

QID                                                                         Q1965979
mid_level_categories                                              [STEM.Mathematics]
tokens                             [демографи, ческая, стати, стика, составная, ч...
mid_level_categories_initial                                      [STEM.Mathematics]
labels                             [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Culture.Language and literature                                                    0
Culture.People                                                                     0
Name: 8900, dtype: object

In [71]:
print("\nRU\n", " ".join(df_wiki_ru.iloc[idx].tokens))


RU
 демографи ческая стати стика составная часть демография система демографических наук системы демографических наук осуществляющих сбор обработку анализ изложение данных характеризующих численность состав размещение движение населения стран территорий отдельных групп населения применением статистика статистических методов совокупность числовых данных численности структуре размещении естественном механическом движении населения область практической деятельности сбору обработке данных демография демографической статистика статистики средние века инки муруа мартин де мартин де муруа своей книге общая история перу сообщал использовании кипу качестве инструмента демографической статистике управлении трудовых ресурсов империя инков империи инков век вели подсч людей селения взрослых детей маркакамайос прибегали помощи кипу назначения людей каждого селения согласно занятию работы демографическая статистика новое время англия родина демографической статистики файл обложка монографии граунта

In [64]:
print("\nEN\n", " ".join(df_language_and_literature_en.iloc[300].tokens))
print("\nRU\n", " ".join(df_language_and_literature_ru.iloc[300].tokens))


EN
 william earl kennard born january one nine one nine five seven american attorney served ambassador european union two zero zero nine one three chairman federal communications commission one nine nine seven two zero zero one nominated ambassador eu president barack obama august two zero zero nine confirmed senate november prior appointment kennard managing director global private equity firm carlyle group also chairman fcc appointed president bill clinton november one nine nine seven serving november three one nine nine seven january one nine two zero zero one early life education kennard graduate stanford university yale law school biography carlyle group prior becoming ambassador kennard managing director carlyle group global private equity firm one zero zero billion management kennard joined carlyle group may two zero zero one led investments telecommunications media sectors work fcc appointment fcc chairman kennard served fcc general counsel one nine nine three one nine nine se

In [74]:
df_language_and_literature_ru.tokens.apply(len).mean()

429.22546673049305

In [75]:
df_wiki_ru.tokens.apply(len).mean()

383.0406060606061

In [77]:
mask = df_wiki_ru.labels.apply(partial(has_label_num, label_num=class2idx["Culture.Entertainment"]))
df_masked_ru = df_wiki_ru[mask == 1]

In [80]:
df_masked_ru
print("\nRU\n", " ".join(df_masked_ru.iloc[300].tokens))


RU
 морская фиалка режисс ра донателлы майорки основе романа джакомо пилати сюжет анжела жив деревне одном итальянских островов отец салваторе занимается добычей туф власти находится весь остров анжела влюбляется подругу детства сару отец слышать желает предосудительной страсти устраивает помолвку дочери местным парнем анжела противится воле отца запирает подвале проводит несколько месяцев заточении мать понимает дочь умереть вынуждает отца устроить подлог деревне объявляется анжела самом деле рождена мальчиком священник ошибся записывая пол понимают это обман противиться властному салваторе никто думает анжела мужском обличии новым именем анжело занимает должность управляющего каменоломнями женится саре счастливый союз гибнет родах сара умирает реб нок чьим биологическим отцом согласию девушек старый приятель сары выживает анжела снимает мужскую одежду нужды скрывать свой пол акт рский состав см также лесбийские отношения искусстве кино лесбийские отношения кинематографе ссылки катег

In [88]:
model.eval()
model(torch.LongTensor([[1,2,3]]), torch.LongTensor([3]))

tensor([[-22.1163,   2.5684, -27.5466, -15.2504, -18.9854, -18.6980, -13.9121,
          -6.4596, -18.1404,   1.9936,  -6.1721, -18.4568, -17.7283, -10.3647,
         -22.9436, -16.7896, -15.9124, -17.0027, -12.3437, -19.7163,   4.5700,
         -24.8053, -22.8432, -22.6332, -37.1863, -16.7402, -18.0485,  -8.5453,
         -17.7292, -15.8727, -15.6236,  -9.8806, -29.0841, -25.3743, -30.2649,
         -24.2633, -28.6702, -12.2714, -23.3339, -20.5276, -22.0185, -15.9751,
         -15.3403, -16.9245]], grad_fn=<AddmmBackward>)

In [90]:
from importlib import reload
reload(utils)

<module 'utils' from '/home/mz2476/topic-modeling/topic-modeling/baseline/utils.py'>

In [97]:
model = dict_model_names["frozen"]["model"]
true, preds = utils.get_true_and_pred_labels(wiki_loaders["train"], model, device, threshold=0.5)

In [107]:
np.array(classes)[true[0].astype(int) == 1]

array(['STEM.Biology'], dtype=object)

In [120]:
np.where(true[:, 7] == 1)[0].shape

(4178,)

In [123]:
preds_for_articles_language_and_literature = preds[np.where(true[:, 7] == 1)[0]]

In [148]:
print("Args sorted:\n", np.argsort(preds_for_articles_language_and_literature.sum(axis=0)))
print("Count sorted:\n", np.sort(preds_for_articles_language_and_literature.sum(axis=0)))

Args sorted:
 [ 0 40 39 38 36 35 34 33 32 26 25 24 22 19 17 21 43  2 12  4  6  8 14  3
  7 37 30 31 42  5 11 27 29 10 23 15  1 28 18 41 16  9 20 13]
Count sorted:
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    1    1    1    2    2    2
    3    3    4    7   14   25   29   42   44   77   79   87  126  174
  747 1257]


In [146]:
classes[20]

'Geography.Europe'

In [156]:
vec_c = true.sum(axis=0)
[(classes[i], count) for i, count in zip(np.argsort(vec_c), np.sort(vec_c))]

[('Geography.Antarctica', 5.0),
 ('STEM.Meteorology', 8.0),
 ('Culture.Media', 8.0),
 ('Culture.Internet culture', 9.0),
 ('Geography.Maps', 10.0),
 ('STEM.Information science', 15.0),
 ('STEM.Mathematics', 23.0),
 ('STEM.Engineering', 29.0),
 ('Culture.Crafts and hobbies', 31.0),
 ('STEM.Time', 46.0),
 ('Geography.Parks', 53.0),
 ('Geography.Landforms', 59.0),
 ('Culture.Arts', 61.0),
 ('STEM.Physics', 67.0),
 ('Geography.Bodies of water', 69.0),
 ('STEM.Chemistry', 86.0),
 ('History_And_Society.Education', 93.0),
 ('Culture.Performing arts', 98.0),
 ('Culture.Food and drink', 106.0),
 ('STEM.Science', 108.0),
 ('STEM.Geosciences', 109.0),
 ('Culture.Plastic arts', 146.0),
 ('Geography.Oceania', 183.0),
 ('STEM.Medicine', 201.0),
 ('Culture.Visual arts', 204.0),
 ('STEM.Space', 211.0),
 ('Culture.Games and toys', 214.0),
 ('Culture.Broadcasting', 221.0),
 ('History_And_Society.Business and economics', 239.0),
 ('Geography.Africa', 288.0),
 ('History_And_Society.Politics and government

In [155]:
vec_c = preds.sum(axis=0)
[(classes[i], count) for i, count in zip(np.argsort(vec_c), np.sort(vec_c))]

[('Culture.Arts', 0),
 ('STEM.Science', 0),
 ('STEM.Physics', 0),
 ('STEM.Meteorology', 0),
 ('STEM.Mathematics', 0),
 ('STEM.Information science', 0),
 ('STEM.Engineering', 0),
 ('History_And_Society.Education', 0),
 ('History_And_Society.Business and economics', 0),
 ('Geography.Maps', 0),
 ('Geography.Antarctica', 0),
 ('Culture.Plastic arts', 0),
 ('Geography.Landforms', 0),
 ('STEM.Time', 0),
 ('Culture.Internet culture', 0),
 ('Culture.Media', 0),
 ('Culture.Crafts and hobbies', 0),
 ('Culture.Language and literature', 1),
 ('Geography.Parks', 2),
 ('Culture.Visual arts', 2),
 ('STEM.Geosciences', 4),
 ('STEM.Chemistry', 4),
 ('Geography.Bodies of water', 6),
 ('Culture.Food and drink', 6),
 ('Culture.Philosophy and religion', 14),
 ('Culture.Entertainment', 14),
 ('History_And_Society.History and society', 18),
 ('Culture.Games and toys', 37),
 ('Culture.Performing arts', 38),
 ('History_And_Society.Transportation', 44),
 ('STEM.Medicine', 80),
 ('Geography.Oceania', 104),
 ('Ge

In [157]:
vec_c = preds_for_articles_language_and_literature.sum(axis=0)
[(classes[i], count) for i, count in zip(np.argsort(vec_c), np.sort(vec_c))]

[('Culture.Arts', 0),
 ('STEM.Science', 0),
 ('STEM.Physics', 0),
 ('STEM.Meteorology', 0),
 ('STEM.Mathematics', 0),
 ('STEM.Information science', 0),
 ('STEM.Geosciences', 0),
 ('STEM.Engineering', 0),
 ('STEM.Chemistry', 0),
 ('History_And_Society.Education', 0),
 ('History_And_Society.Business and economics', 0),
 ('Geography.Parks', 0),
 ('Geography.Maps', 0),
 ('Geography.Bodies of water', 0),
 ('Geography.Antarctica', 0),
 ('Geography.Landforms', 0),
 ('STEM.Time', 0),
 ('Culture.Crafts and hobbies', 0),
 ('Culture.Plastic arts', 0),
 ('Culture.Food and drink', 0),
 ('Culture.Internet culture', 0),
 ('Culture.Media', 0),
 ('Culture.Visual arts', 1),
 ('Culture.Entertainment', 1),
 ('Culture.Language and literature', 1),
 ('STEM.Medicine', 2),
 ('History_And_Society.Transportation', 2),
 ('STEM.Biology', 2),
 ('STEM.Technology', 3),
 ('Culture.Games and toys', 3),
 ('Culture.Philosophy and religion', 4),
 ('History_And_Society.History and society', 7),
 ('History_And_Society.Poli

In [130]:

np.array(classes)[None, :] * preds_for_articles_language_and_literature

array([['', '', '', ..., '', '', ''],
       ['', '', '', ..., '', '', ''],
       ['', '', '', ..., '', '', ''],
       ...,
       ['', '', '', ..., '', '', ''],
       ['', '', '', ..., '', '', ''],
       ['', '', '', ..., '', '', '']], dtype=object)

In [113]:
preds[np.where(true[:, 7] == 1)]

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [105]:
true[0].astype(int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])